In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Data preparation
dataset_dir = 'dataset'  # Path to the folder containing positive and negative folders

# Image data generator with validation split
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of data for validation
)

# Training data generator
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Specify training subset
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation',  # Specify validation subset
    shuffle=False  # Important for metrics calculation
)

# Load MobileNetV2 as base model with error handling for pre-trained weights
try:
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
except Exception as e:
    print(f"Error loading pre-trained weights: {e}")
    base_model = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout for regularization
predictions = Dense(1, activation='sigmoid')(x)

# Model creation
model = Model(inputs=base_model.input, outputs=predictions)

# Model compilation
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=5,  # Adjust epochs as needed
    validation_data=validation_generator
)

# Save the model
model.save('deep_crack_model.h5')  

# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")

# Generate predictions for the validation set
Y_pred = model.predict(validation_generator)
Y_pred_classes = np.round(Y_pred).flatten()
Y_true = validation_generator.classes

# Calculate additional metrics
accuracy = accuracy_score(Y_true, Y_pred_classes)
precision = precision_score(Y_true, Y_pred_classes)
recall = recall_score(Y_true, Y_pred_classes)
f1 = f1_score(Y_true, Y_pred_classes)

# Print metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Classification report
print("\nClassification Report:")
print(classification_report(Y_true, Y_pred_classes, target_names=validation_generator.class_indices.keys()))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(Y_true, Y_pred_classes))
 

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/5
50/50 [==============================] - 59s 1s/step - loss: 0.1975 - accuracy: 0.9281 - val_loss: 0.1001 - val_accuracy: 0.9600
Epoch 2/5
50/50 [==============================] - 45s 897ms/step - loss: 0.0985 - accuracy: 0.9737 - val_loss: 0.0528 - val_accuracy: 0.9800
Epoch 3/5
50/50 [==============================] - 44s 880ms/step - loss: 0.0842 - accuracy: 0.9762 - val_loss: 0.0470 - val_accuracy: 0.9875
Epoch 4/5
50/50 [==============================] - 51s 1s/step - loss: 0.0610 - accuracy: 0.9837 - val_loss: 0.0286 - val_accuracy: 0.9975
Epoch 5/5
13/13 [==============================] - 9s 666ms/step - loss: 0.0540 - accuracy: 0.9800
Validation Loss: 0.05
Validation Accuracy: 0.98
Accuracy: 0.98
Precision: 0.98
Recall: 0.99
F1-Score: 0.99

Classification Report:
              precision    recall  f1-score   support

    Negative       0.99      0.97      0.98       200
    Positive    

In [4]:
pip install --upgrade tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\LENOVO\\AppData\\Local\\Programs\\Python\\Python38\\Lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
  Using cached tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl.metadata (4.1 kB)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached protobuf-4.25.6-cp38-cp38-win_amd64.whl.metadata (541 bytes)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.13.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached tensorboard_data_server-0.7.2-p

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Layer, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Custom Bilinear Interpolation Layer
class BilinearInterpolationLayer(Layer):
    def __init__(self, target_size=(7, 7), **kwargs):
        super(BilinearInterpolationLayer, self).__init__(**kwargs)
        self.target_size = target_size

    def call(self, inputs):
        # Resizing spatial dimensions using bilinear interpolation
        return tf.image.resize(inputs, self.target_size, method='bilinear')

# Data preparation
dataset_dir = 'dataset'  # Path to the folder containing 'positive' and 'negative' folders

# Image data generator with validation split
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% of data for validation
)

# Training data generator
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Specify training subset
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation',  # Specify validation subset
    shuffle=False  # Important for metrics calculation
)

# Load ResNet50 as the base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Custom top layers
x = base_model.output  # Feature maps from ResNet50
x = BilinearInterpolationLayer(target_size=(7, 7))(x)  # Apply bilinear interpolation
x = GlobalAveragePooling2D()(x)  # Reduce to 1D
x = Dense(128, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Regularization

predictions = Dense(1, activation='sigmoid')(x)

# Model creation
model = Model(inputs=base_model.input, outputs=predictions)

# Model compilation
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=5,  # Adjust epochs as needed
    validation_data=validation_generator
)

# Save the model
model.save('rfcn_b_model.h5')

# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")

# Generate predictions for the validation set
Y_pred = model.predict(validation_generator)
Y_pred_classes = np.round(Y_pred).flatten()
Y_true = validation_generator.classes

# Calculate additional metrics
accuracy = accuracy_score(Y_true, Y_pred_classes)
precision = precision_score(Y_true, Y_pred_classes)
recall = recall_score(Y_true, Y_pred_classes)
f1 = f1_score(Y_true, Y_pred_classes)

# Print metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Classification report
print("\nClassification Report:")
print(classification_report(Y_true, Y_pred_classes, target_names=validation_generator.class_indices.keys()))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(Y_true, Y_pred_classes))


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/5
50/50 [==============================] - 397s 8s/step - loss: 0.7129 - accuracy: 0.5144 - val_loss: 0.6815 - val_accuracy: 0.5000
Epoch 2/5
50/50 [==============================] - 396s 8s/step - loss: 0.6868 - accuracy: 0.5606 - val_loss: 0.6921 - val_accuracy: 0.5000
Epoch 3/5
50/50 [==============================] - 417s 8s/step - loss: 0.6817 - accuracy: 0.5581 - val_loss: 0.6804 - val_accuracy: 0.5225
Epoch 4/5
50/50 [==============================] - 367s 7s/step - loss: 0.6765 - accuracy: 0.5888 - val_loss: 0.6722 - val_accuracy: 0.6825
Epoch 5/5
50/50 [==============================] - 387s 8s/step - loss: 0.6707 - accuracy: 0.5825 - val_loss: 0.6594 - val_accuracy: 0.7475


c:\Users\THANK YOU\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - 66s 5s/step - loss: 0.6592 - accuracy: 0.7225
Validation Loss: 0.66
Validation Accuracy: 0.72
13/13 [==============================] - 69s 5s/step
Accuracy: 0.74
Precision: 0.82
Recall: 0.61
F1-Score: 0.70

Classification Report:
              precision    recall  f1-score   support

    Negative       0.69      0.86      0.77       200
    Positive       0.82      0.61      0.70       200

    accuracy                           0.74       400
   macro avg       0.75      0.74      0.73       400
weighted avg       0.75      0.74      0.73       400


Confusion Matrix:
[[173  27]
 [ 78 122]]


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

# Path to Dataset
dataset_dir = "dataset"  # Replace with the path to your dataset

# Data Generators for Training and Validation
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize images to [0, 1]
    validation_split=0.2  # Use 20% of the data for validation
)

# Training Data Generator
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # Resize images to the input size of the model
    batch_size=32,
    class_mode='binary',  # Binary classification (positive vs. negative)
    subset='training'
)

# Validation Data Generator
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Backbone (Feature Extractor)
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)
backbone = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

# Final Classification Layer
x = GlobalAveragePooling2D()(backbone.output)
final_class = Dense(1, activation='sigmoid', name='final_class_output')(x)  # Sigmoid for binary classification

# Model
model = Model(inputs, final_class)

# Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',  # Binary classification
    metrics=['accuracy']  # Accuracy metric
)

# Train the Model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# Save the trained model
model.save("binary_classification_model.h5")

# Evaluate the Model
val_loss, val_accuracy = model.evaluate(validation_generator)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Generate Predictions
Y_pred = model.predict(validation_generator)
Y_pred_classes = (Y_pred > 0.5).astype(int).flatten()  # Convert predictions to binary values (0 or 1)
Y_true = validation_generator.classes

# Calculate Metrics
accuracy = accuracy_score(Y_true, Y_pred_classes)
recall = recall_score(Y_true, Y_pred_classes)
precision = precision_score(Y_true, Y_pred_classes)
f1 = f1_score(Y_true, Y_pred_classes)

# Print Metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1-Score: {f1:.2f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(Y_true, Y_pred_classes, target_names=validation_generator.class_indices.keys()))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(Y_true, Y_pred_classes))


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/5
50/50 [==============================] - 16974s 338s/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 101571.2734 - val_accuracy: 0.5000
Epoch 2/5
50/50 [==============================] - 39929s 814s/step - loss: 0.1382 - accuracy: 0.9588 - val_loss: 1.0708 - val_accuracy: 0.5000
Epoch 3/5
50/50 [==============================] - 1112s 22s/step - loss: 0.0912 - accuracy: 0.9681 - val_loss: 0.7153 - val_accuracy: 0.5000
Epoch 4/5
50/50 [==============================] - 1053s 21s/step - loss: 0.1106 - accuracy: 0.9669 - val_loss: 0.7802 - val_accuracy: 0.5000
Epoch 5/5
50/50 [==============================] - 1151s 23s/step - loss: 0.0648 - accuracy: 0.9781 - val_loss: 0.9587 - val_accuracy: 0.5000


c:\Users\THANK YOU\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - 53s 4s/step - loss: 0.9587 - accuracy: 0.5000
Validation Loss: 0.9586554765701294
Validation Accuracy: 0.5
13/13 [==============================] - 56s 4s/step
Accuracy: 0.50
Recall: 0.00
Precision: 0.00
F1-Score: 0.00

Classification Report:
              precision    recall  f1-score   support

    Negative       0.50      1.00      0.67       200
    Positive       0.00      0.00      0.00       200

    accuracy                           0.50       400
   macro avg       0.25      0.50      0.33       400
weighted avg       0.25      0.50      0.33       400


Confusion Matrix:
[[200   0]
 [200   0]]


c:\Users\THANK YOU\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\THANK YOU\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\THANK YOU\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [31]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('improved_model.h5')

# Function to preprocess the input image
def preprocess_image(image_path):
    """
    Preprocess the image for prediction.
    Args:
        image_path (str): Path to the image to predict.
    Returns:
        np.array: Preprocessed image ready for the model.
    """
    img = load_img(image_path, target_size=(224, 224))  # Resize the image
    img_array = img_to_array(img)  # Convert to array
    img_array = img_array / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make a prediction
def predict_image(image_path):
    """
    Predict whether the image is a crack or non-crack.
    Args:
        image_path (str): Path to the image to predict.
    Returns:
        str: Predicted class ('Crack' or 'Non-Crack').
    """
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    
    if prediction[0] >= 0.3:
        return "Crack"
    else:
        return "Non-Crack"

# Example usage
image_path = "dataset/Positive/00097.jpg"  # Replace with your image path
result = predict_image(image_path)
print(f"The prediction for the input image is: {result}")


1/1 [==============================] - 9s 9s/step
The prediction for the input image is: Crack
